# **Hard level**

Принадлежность идеала и его порождающий в кольцах главных идеалов

**В кольце Z**: идеал, порождаемый любым конечным набором чисел, совпадает с идеалом, порожденным их НОД. Соответственно, любой идеал в Z - главный.
Найдём НОД по условию.

In [2]:
def basic_gcd(arg1: int, arg2: int) -> int:
    """
    Находит НОД алгоритмом Евклида
    """
    while arg1 != 0:
        arg1, arg2 = arg2 % arg1, arg1
    return arg2


def find_d(*args) -> int:
    """
    Находит d из условия
    """
    if len(args) == 1:
        return args[0]
    if len(args) == 0:
        raise ValueError("enter valid amount of arguments")

    result = args[0]
    for num in args[1:]:
        result = basic_gcd(result, num)
    return result


inp = input("Enter numbers separated by spaces: ")
numbers = [int(x) for x in inp.split()]
print('d = ', find_d(*numbers))

Enter numbers separated by spaces:  35 63 84


d =  7


**В K[x]**: идеал, порождаемый любым конечным набором чисел, совпадает с идеалом, порожденным их НОД. Соответственно, любой идеал в Z - главный.
Найдём НОД по условию.

In [ ]:
# К слову, если делать по образцу, то мой код для обработки всех запросов получается примерно таким, как снизу.
# Сверху оставляю базовую версию для кольца Z.

In [1]:
# from typing import List, Union


# ----------------- Класс элемента кольца -----------------
class RingElement:
    """Хранит либо целое число, либо полином (список коэффициентов: [a0, a1, ..., an])."""
    def __init__(self, data: Union[int, List[float]]):
        if isinstance(data, int):
            self.data = data
            self.is_poly = False
        elif isinstance(data, list):
            # Убираем лишние нули в конце (например, [1,2,0,0] → [1,2])
            while len(data) > 1 and abs(data[-1]) < 1e-10:
                data.pop()
            # Если остался только ноль — оставляем [0]
            if not data or (len(data) == 1 and abs(data[0]) < 1e-10):
                data = [0.0]
            self.data = data
            self.is_poly = True
        else:
            raise ValueError("RingElement принимает int или list[float]")

    def __repr__(self):
        if not self.is_poly:
            return str(self.data)
        # Простое отображение полинома: 2 + -3*x + 1*x^2 → улучшим чуть-чуть
        if self.data == [0.0]:
            return "0"
        parts = []
        for i, c in enumerate(self.data):
            if abs(c) < 1e-10:
                continue
            if i == 0:
                parts.append(f"{c}")
            elif i == 1:
                parts.append(f"{c}*x")
            else:
                parts.append(f"{c}*x^{i}")
        return " + ".join(parts).replace("+ -", "- ")


# ----------------- Вспомогательные функции для полиномов -----------------

def poly_trim(p):
    """Обрезает ведущие нули. Возвращает [0.0], если полином нулевой."""
    p = p[:]
    while len(p) > 1 and abs(p[-1]) < 1e-10:
        p.pop()
    if not p or (len(p) == 1 and abs(p[0]) < 1e-10):
        return [0.0]
    return p


def poly_divmod(num, den):
    """
    Делит num на den с остатком: num = den * q + r.
    Работает над полем (деление на ведущий коэффициент разрешено).
    Возвращает (частное, остаток).
    """
    num = poly_trim(num[:])
    den = poly_trim(den[:])

    if den == [0.0]:
        raise ZeroDivisionError("Деление полинома на ноль")

    if num == [0.0]:
        return [0.0], [0.0]

    deg_num = len(num) - 1
    deg_den = len(den) - 1
    lc_den = den[-1]  # ведущий коэффициент делителя

    if deg_num < deg_den:
        return [0.0], num

    q = [0.0] * (deg_num - deg_den + 1)
    r = num[:]  # копия делимого

    while len(r) - 1 >= deg_den and r != [0.0]:
        deg_r = len(r) - 1
        if deg_r < deg_den:
            break
        # На сколько умножить делитель, чтобы "сбить" старший член остатка
        coeff = r[-1] / lc_den
        shift = deg_r - deg_den
        q[shift] = coeff

        # Формируем den * coeff * x^shift
        term = [0.0] * shift + [c * coeff for c in den]
        # Вычитаем из r
        r = poly_trim([r[i] - (term[i] if i < len(term) else 0.0) for i in range(max(len(r), len(term)))])
    
    return poly_trim(q), r


def poly_gcd(p, q):
    """НОД двух полиномов. Возвращает монический (старший коэффициент = 1), если не ноль."""
    p, q = poly_trim(p[:]), poly_trim(q[:])

    # Алгоритм Евклида
    while q != [0.0]:
        _, r = poly_divmod(p, q)
        p, q = q, r

    if p == [0.0]:
        return [0.0]

    # Нормируем: делим на старший коэффициент → делаем ведущий коэффициент = 1
    lc = p[-1]
    if abs(lc) > 1e-10:
        p = [c / lc for c in p]
    return poly_trim(p)


# ----------------- Основная функция: НОД списка элементов -----------------

def gcd_ring_elements(elements: List[RingElement]) -> RingElement:
    """Находит порождающий идеала (a1, ..., ak). Возвращает d, такой что (a1,...,ak) = (d)."""
    if not elements:
        raise ValueError("Список элементов не может быть пустым")

    # Работаем с целыми числами
    if not elements[0].is_poly:
        nums = [abs(el.data) for el in elements]  # НОД считаем по модулям
        # Стартуем с 0: gcd(0, a) = a
        d = 0
        for n in nums:
            while n != 0:
                d, n = n, d % n
        return RingElement(d)

    # Работаем с полиномами
    polys = [el.data[:] for el in elements]
    current = polys[0]
    for p in polys[1:]:
        current = poly_gcd(current, p)
        if current == [0.0]:
            break
    return RingElement(current)


# ----------------- Проверка принадлежности элемента идеалу -----------------

def in_ideal(element: RingElement, gen: RingElement) -> bool:
    """Проверяет, делится ли element на gen — т.е. лежит ли element в (gen)."""
    if element.is_poly != gen.is_poly:
        raise TypeError("Элемент и генератор должны быть одного типа")

    if not element.is_poly:
        a, d = element.data, gen.data
        return a == 0 if d == 0 else (a % d == 0)

    # Для полиномов — проверяем остаток от деления
    f, g = element.data, gen.data
    if g == [0.0]:
        return f == [0.0]
    _, r = poly_divmod(f, g)
    return r == [0.0]


# ----------------- Основная программа: ввод и вывод -----------------

def parse_input():
    """Спрашивает у юзера: числа или полиномы? Возвращает список RingElement."""
    print("Выберите кольцо:")
    print("1 — целые числа (ℤ)")
    print("2 — полиномы над полем (K[x])")
    choice = input("Ваш выбор (1 или 2): ").strip()

    if choice == "1":
        line = input("Введите целые числа через пробел: ").strip()
        if not line:
            raise ValueError("Ничего не введено")
        nums = [int(x) for x in line.split()]
        return [RingElement(n) for n in nums]

    elif choice == "2":
        print("Введите полиномы по одному. Коэффициенты через пробел: a0 a1 ... an -> a0 + a1*x + ... + an*x^n")
        print("Пример: '1 -2 1' означает 1 - 2*x + 1*x^2 = (x-1)^2")
        print("Пустая строка — конец ввода.")
        polys = []
        while True:
            line = input("Полином (или Enter для завершения): ").strip()
            if not line:
                break
            try:
                coeffs = [float(x) for x in line.split()]
                if not coeffs:
                    continue
                polys.append(RingElement(coeffs))
            except Exception as e:
                print(f"Ошибка ввода: {e}. Попробуйте снова.")
        if not polys:
            raise ValueError("Нужно ввести хотя бы один полином")
        return polys

    else:
        raise ValueError("Неверный выбор: введите 1 или 2")


# ----------------- Запуск -----------------

if __name__ == "__main__":
    try:
        elements = parse_input()
        print("\nВы ввели:")
        for i, el in enumerate(elements, 1):
            print(f"  {i}) {el}")

        d = gcd_ring_elements(elements)
        print(f"\nПорождающий элемент идеала: {d}")
        print(f"Идеал = ({d})")

        # Дополнительно: спросим, проверить ли принадлежность?
        print("\nХотите проверить, принадлежит ли какой-то элемент этому идеалу?")
        ans = input("Да/нет (по умолчанию — да): ").strip().lower()
        if ans in ("", "да", "yes", "y"):
            if not elements[0].is_poly:
                try:
                    a = int(input("Введите целое число: "))
                    elem = RingElement(a)
                except:
                    print("Неверное число — пропускаем проверку.")
                    elem = None
            else:
                line = input("Введите коэффициенты полинома (a0 a1 ...): ").strip()
                try:
                    coeffs = [float(x) for x in line.split()]
                    elem = RingElement(coeffs)
                except:
                    print("Ошибка ввода — пропускаем проверку.")
                    elem = None

            if elem is not None:
                result = in_ideal(elem, d)
                print(f"\n{elem} {'∈' if result else '∉'} ({d})")

    except Exception as e:
        print(f"Ошибка: {e}")

Выберите кольцо:
1 — целые числа (ℤ)
2 — полиномы над полем (K[x])


Ваш выбор (1 или 2):  1
Введите целые числа через пробел:  4 7 9 45



Вы ввели:
  1) 4
  2) 7
  3) 9
  4) 45

Порождающий элемент идеала: 1
Идеал = (1)

Хотите проверить, принадлежит ли какой-то элемент этому идеалу?


Да/нет (по умолчанию — да):  Да



Введите элемент для проверки принадлежности идеалу (1):


Введите целое число:  67



67 ∈ (1)
Так как 67 = 1 × 67

Программа завершена.


# **Expert level**

**Задача (Expert): Докажите или опровергните утверждение:**

Приведённые примеры порядков – отношения порядка на мономах.

Пусть $ R = K[x_1, \dots, x_n] $,  
$ \mathcal{M} = \{ x_1^{a_1} \cdots x_n^{a_n} \mid \mathbf{a} = (a_1,\dots,a_n) \in \mathbb{Z}_{\ge 0}^n \} $,  
$ |\mathbf{a}| = a_1 + \cdots + a_n $.

Отношение $ \prec $ на $ \mathcal{M} $ — *порядок на мономах*, если:
1. $ a \prec b,\ b \prec c \;\Rightarrow\; a \prec c $,
2. $ \forall a,b:\ a = b $ или $ a \prec b $ или $ b \prec a $,
3. $ a \prec b \;\Rightarrow\; ac \prec bc $ для любого $ c \in \mathcal{M} $.

---

**lex:**  
$ \mathbf{a} \prec_{\mathrm{lex}} \mathbf{b} \iff \exists i:\ a_1 = b_1, \dots, a_{i-1} = b_{i-1},\ a_i < b_i $.

- Транзитивность и линейность — по определению лексикографического сравнения.
- Мультипликативность: если $ \mathbf{a} \prec_{\mathrm{lex}} \mathbf{b} $, то $ \mathbf{a} + \mathbf{c} \prec_{\mathrm{lex}} \mathbf{b} + \mathbf{c} $, так как сдвиг на $ \mathbf{c} $ не меняет первую координату различия.

→ `lex` удовлетворяет (1)–(3).

---

**grlex:**  
$ \mathbf{a} \prec_{\mathrm{grlex}} \mathbf{b} \iff 
|\mathbf{a}| < |\mathbf{b}| $ или $ (|\mathbf{a}| = |\mathbf{b}| $ и $ \mathbf{a} \prec_{\mathrm{lex}} \mathbf{b}) $.

- Транзитивность: проверяется разбором случаев по $ |\cdot| $ и использованием транзитивности `lex`.
- Линейность: сравнение сначала по $ |\cdot| $, затем по `lex`.
- Мультипликативность:  
  $ |\mathbf{a} + \mathbf{c}| = |\mathbf{a}| + |\mathbf{c}| $,  
  поэтому $ |\mathbf{a}| < |\mathbf{b}| \Rightarrow |\mathbf{a}+\mathbf{c}| < |\mathbf{b}+\mathbf{c}| $,  
  а если $ |\mathbf{a}| = |\mathbf{b}| $, то $ \mathbf{a} \prec_{\mathrm{lex}} \mathbf{b} \Rightarrow \mathbf{a}+\mathbf{c} \prec_{\mathrm{lex}} \mathbf{b}+\mathbf{c} $.

→ `grlex` удовлетворяет (1)–(3).

---

Оба — мономиальные порядки.

---
**Задача (Expert): Дайте ответы на вопросы. Что же это
значит. Как нам интерпретировать результаты ритуала?**

• если получился 0?

• если получился не 0?

• существует ли порядочный подданный, который не сможет пройти ритуал?

• зависит ли результат ритуала от того, в каком порядке
инквизиторы будут проводить ритуал?

---

1. Если в результате полином редуцирован до 0, то он гарантированно попадает в идеал.

2. Если в результате получился не 0, то полином может принадлежать идеалу, а может не принадлежать.

3. Да, существует. Так происходит из-за потенциального несовпадения идеала, порождаемого старшими членами порождающих полиномов и идеала "подданого" (старших членов всего I).

Пример.
Пусть $K=Q$, $G={g_1,g_2}$ , где $g_1 = x^2−y$, $g_2 = xy - 1$

Мономиальный порядок: $lex, x>y$ .
Тогда:

$LM(g_1) = x^2$, $LT(g_1) = x^2$  
$LM(g_2) = xy$, $LT(g_2) = xy$

Построим конкретный $f \in I$ 

Рассмотрим линейную комбинацию:
$$f = y \cdot g_1 - x \cdot g_2$$

Это просто элемент идеала, так как он является $R$-линейной комбинацией порождающих.

Вычислим его:
$$
\begin{aligned}
f &= y(x^2 - y) - x(xy - 1) \\
&= x^2y - y^2 - x^2y + x \\
&= x - y^2
\end{aligned}
$$

Пусть $f = x - y^2$

Проведём редукцию $f$ по $G = \{g_1, g_2\}$

$$f = x - y^2 \neq 0$$

Старший моном: при $lex$, $x > y$, имеем
$$LM(f) = x, \quad LT(f) = x$$

Теперь смотрим на ведущие мономы порождающих:
$$LM(g_1) = x^2, \quad LM(g_2) = xy$$

Делит ли какой-нибудь из них $LM(f) = x$?

- $x^2 \mid x$? → **Нет**, степень по $x$ выше
  
- $xy \mid x$? → **Нет**, в $xy$ есть $y$, а в $x$ — нет

  
- $LM(f) = x$ не делится ни на $x^2$, ни на $xy$

- Редукция невозможна: $f_G = f \neq 0$

Получается, $f \in I$, но редукция не приводит к 0.

4. В общем случае результат редукции зависит от порядка выбора редукторов (т.е. от того, какой $g_i$ применяется на каждом шаге), если множество $G={g_1,…,g_m}$ не является базисом Грёбнера.

---

**Задача (Expert): Любой ли набор полиномов – базис Грёбнера?**


Нет, не любой. Достаточно посмотреть на примеры с "изгнанными". 

Для произвольного конечного набора полиномов  $G = \{g_1, \dots, g_m\} \subset K[x_1, \dots, x_n]$ в общем случае идеал, порождённый старшими членами порождающих, строго содержится в идеале старших членов всего идеала.

Равенство
${in}(I) = \langle \operatorname{in}(g_1), \dots, \operatorname{in}(g_m) \rangle$
выполняется тогда и только тогда, когда $\{g_1, \dots, g_m\}$ является базисом Грёбнера идеала $I$ относительно заданного мономиального порядка.

---

**Задача (Expert): Дайте ответы на вопросы. Что если
провести ритуал с базисом Грёбнера вместо артефактов? Как
теперь интерпретировать результаты?**


• если получился 0?   <<< так же

• если получился не 0?    <<< теперь исключены ложно-отрицательные случаи

• существует ли порядочный подданный, который не сможет пройти ритуал?   <<< нет

• зависит ли результат ритуала от того, в каком порядке инквизиторы будут проводить ритуал? <<< нет

Пусть теперь *артефакты* $G = \{g_1, \dots, g_m\}$ образуют базис Грёбнера идеала  
$I = \langle g_1, \dots, g_m \rangle$ относительно фиксированного мономиального порядка.

Тогда ритуал редукции обретает детерминированность и полноту:

- Если редукция даёт $0$ 
  $\;\Rightarrow\; f \in I$.  
  (Старшие члены покрывают идеал полностью — очищение завершено.)

- Если редукция даёт $r \ne 0$ 
  $\;\Rightarrow\; f \notin I$.  
  (Остаток $r$ — канонический представитель класса $f + I$ в $K[x_1,\dots,x_n]/I$.)

- Существует ли «порядочный подданный» ($f \in I$), не проходящий ритуал? 
  *Нет.* Для любого $f \in I$ нормальная форма $\overline{f}^G = 0$.  
  (Базис Грёбнера гарантирует: $\operatorname{in}(I) = \langle \operatorname{in}(g_1),\dots,\operatorname{in}(g_m) \rangle$.)

- Зависит ли результат от порядка редукций?
  *Нет.* Нормальная форма $\overline{f}^G$ *единственна*, независимо от выбора $g_i$ на каждом шаге.  

---

**Задача (Expert): Докажите или опровергните утверждение:
Любой базис Грёбнера можно привести к редуцированному.**

**Конструктивное доказательство через алгоритм приведения**

Пусть $G = \{g_1, \dots, g_m\}$ — произвольный базис Грёбнера идеала $I$. Построим редуцированный базис $\widetilde{G}$.

**1. Нормализация**

Заменим каждый $g_i$ на

$$
h_i = \frac{1}{\mathrm{LC}(g_i)} \cdot g_i
$$

Тогда $\mathrm{LC}(h_i) = 1$, и $H = \{h_1, \dots, h_m\}$ — тоже базис Грёбнера (ведущие мономы не изменились).


**2. Полная редукция**

Упорядочим $H$ так, чтобы $\mathrm{LM}(h_1) > \mathrm{LM}(h_2) > \cdots > \mathrm{LM}(h_m)$.

Последовательно (от $i = m$ до $1$) выполним:

Приведём $h_i$ к нормальной форме относительно множества $\{h_1, \dots, h_{i-1}\}$:

$$
\widetilde{h}_i = \mathrm{NF}(h_i \mid \{h_1, \dots, h_{i-1}\})
$$

Поскольку $H$ — базис Грёбнера, эта нормальная форма существует и единственна; при этом $\mathrm{LM}(\widetilde{h}_i) = \mathrm{LM}(h_i)$ (иначе мы потеряли бы покрытие $\mathrm{in}(I)$).

Положим $\widetilde{G} = \{\widetilde{h}_1, \dots, \widetilde{h}_m\}$.



**Проверка свойств:**

1. $\mathrm{LC}(\widetilde{h}_i) = 1$ (редукция не меняет ведущий коэффициент, если он $1$ и не участвует в сокращении)
2. Каждый $\widetilde{h}_i$ редуцирован относительно остальных (по построению)
3. $\widetilde{G}$ порождает тот же идеал (каждая редукция — вычитание элемента из $I$)
4. $\mathrm{LM}(\widetilde{h}_i) = \mathrm{LM}(h_i)$, значит, $\langle \mathrm{in}(\widetilde{G}) \rangle = \langle \mathrm{in}(H) \rangle = \mathrm{in}(I)$, т.е. $\widetilde{G}$ — базис Грёбнера

Следовательно, $\widetilde{G}$ — редуцированный базис Грёбнера.

**Задача (Expert): Реализуйте алгоритм Бухбергера с
оптимизациями** + требования

In [2]:
"""
Реализация алгоритма Бухбергера с оптимизацией - критерий взаимной простоты.
Если для всех i != j мономы LM(fi) и LM(fj) взаимно просты,
то {f1, ..., fm} уже является базисом Грёбнера.
"""

from typing import Dict, List, Tuple, Optional, Callable
from itertools import combinations


# Типы
Monomial = Tuple[int, ...]
Coefficient = float
Polynomial = Dict[Monomial, Coefficient]


class RingElement:
    """Полином над полем, представленный как словарь {моном: коэффициент}."""
    
    def __init__(self, terms: Polynomial = None, n_vars: int = 2):
        self.terms = self._trim(terms or {})
        self.n_vars = n_vars

    @staticmethod
    def _trim(poly: Polynomial) -> Polynomial:
        """Удаляет члены с нулевыми коэффициентами."""
        cleaned = {m: c for m, c in poly.items() if abs(c) > 1e-12}
        return cleaned if cleaned else {tuple(): 0.0}

    def is_zero(self) -> bool:
        return all(abs(c) < 1e-12 for c in self.terms.values())

    def copy(self) -> "RingElement":
        return RingElement(self.terms.copy(), self.n_vars)

    def __sub__(self, other: "RingElement") -> "RingElement":
        terms = self.terms.copy()
        for m, c in other.terms.items():
            terms[m] = terms.get(m, 0.0) - c
        return RingElement(terms, self.n_vars)

    def __repr__(self) -> str:
        if self.is_zero():
            return "0"
        # Сортировка для стабильного вывода
        items = sorted(
            [(m, c) for m, c in self.terms.items() if abs(c) > 1e-12],
            key=lambda mc: (-sum(mc[0]), mc[0])
        )
        parts = []
        for m, c in items:
            var_part = "".join(
                f"x{i+1}" + (f"^{e}" if e > 1 else "")
                for i, e in enumerate(m) if e > 0
            ) or "1"
            if var_part == "1":
                term = f"{c:g}"
            elif abs(c - 1.0) < 1e-10:
                term = var_part
            elif abs(c + 1.0) < 1e-10:
                term = "-" + var_part
            else:
                term = f"{c:g}*{var_part}"
            parts.append(term)
        return " + ".join(parts).replace("+ -", "- ")


# Вспомогательные функции для мономов

def monomial_lcm(m1: Monomial, m2: Monomial) -> Monomial:
    return tuple(max(a, b) for a, b in zip(m1, m2))

def monomial_divides(divisor: Monomial, dividend: Monomial) -> bool:
    return all(a <= b for a, b in zip(divisor, dividend))

def monomial_div(dividend: Monomial, divisor: Monomial) -> Monomial:
    return tuple(b - a for a, b in zip(divisor, dividend))


# Основные функции

def leading_monomial(f: RingElement, order_key) -> Optional[Monomial]:
    non_zero_monomials = [m for m, c in f.terms.items() if abs(c) > 1e-12]
    return max(non_zero_monomials, key=order_key) if non_zero_monomials else None

def leading_term(f: RingElement, order_key) -> Optional[Tuple[Monomial, float]]:
    lm = leading_monomial(f, order_key)
    return (lm, f.terms[lm]) if lm is not None else None


def reduce_one_step(f: RingElement, g: RingElement, order_key) -> Optional[RingElement]:
    """Выполняет один шаг редукции f с помощью g."""
    lt_f = leading_term(f, order_key)
    lt_g = leading_term(g, order_key)
    if lt_f is None or lt_g is None:
        return None
    lm_f, lc_f = lt_f
    lm_g, lc_g = lt_g
    if not monomial_divides(lm_g, lm_f):
        return None
    coeff = lc_f / lc_g
    mon_mult = monomial_div(lm_f, lm_g)
    # Построение g * coeff * x^mon_mult
    new_terms = {}
    for m, c in g.terms.items():
        l = max(len(m), len(mon_mult))
        m_ext = m + (0,) * (l - len(m))
        mult_ext = mon_mult + (0,) * (l - len(mon_mult))
        new_m = tuple(a + b for a, b in zip(mult_ext, m_ext))
        new_terms[new_m] = new_terms.get(new_m, 0.0) + c * coeff
    return f - RingElement(new_terms, f.n_vars)


def normal_form(f: RingElement, basis: List[RingElement], order_key) -> RingElement:
    """Вычисляет нормальную форму f относительно basis."""
    h = f.copy()
    while True:
        reduced = False
        for g in basis:
            h_new = reduce_one_step(h, g, order_key)
            if h_new is not None:
                h = h_new
                reduced = True
                break
        if not reduced or h.is_zero():
            break
    return h


def S_polynomial(f: RingElement, g: RingElement, order_key) -> RingElement:
    """Строит S-полином от f и g."""
    lt_f = leading_term(f, order_key)
    lt_g = leading_term(g, order_key)
    if lt_f is None or lt_g is None:
        return RingElement()
    lm_f, lc_f = lt_f
    lm_g, lc_g = lt_g
    L = monomial_lcm(lm_f, lm_g)
    # Множители: m_f = L / in(f), m_g = L / in(g)
    coeff_f = 1.0 / lc_f
    mon_f = monomial_div(L, lm_f)
    coeff_g = 1.0 / lc_g
    mon_g = monomial_div(L, lm_g)
    # m_f * f
    terms_f = {}
    for m, c in f.terms.items():
        l = max(len(m), len(mon_f))
        m_pad = m + (0,) * (l - len(m))
        mf_pad = mon_f + (0,) * (l - len(mon_f))
        new_m = tuple(a + b for a, b in zip(mf_pad, m_pad))
        terms_f[new_m] = terms_f.get(new_m, 0.0) + c * coeff_f
    # m_g * g
    terms_g = {}
    for m, c in g.terms.items():
        l = max(len(m), len(mon_g))
        m_pad = m + (0,) * (l - len(m))
        mg_pad = mon_g + (0,) * (l - len(mon_g))
        new_m = tuple(a + b for a, b in zip(mg_pad, m_pad))
        terms_g[new_m] = terms_g.get(new_m, 0.0) + c * coeff_g
    return RingElement(terms_f) - RingElement(terms_g)


# Критерий взаимной простоты

def are_coprime(m1: Monomial, m2: Monomial) -> bool:
    """
    Проверяет, являются ли два монома взаимно простыми:
    не имеют общих переменных с положительной степенью.
    """
    for a, b in zip(m1, m2):
        if a > 0 and b > 0:
            return False
    return True


def coprime_criterion_applies(f: RingElement, g: RingElement, order_key) -> bool:
    """
    Применим ли критерий взаимной простоты к паре (f, g)?
    Возвращает True, если LM(f) и LM(g) взаимно просты.
    """
    lm_f = leading_monomial(f, order_key)
    lm_g = leading_monomial(g, order_key)
    if lm_f is None or lm_g is None:
        return False
    return are_coprime(lm_f, lm_g)


# Алгоритм Бухбергера с оптимизацией

def buchberger(f_list: List[RingElement], order_key) -> List[RingElement]:
    """
    Вычисляет редуцированный базис Грёбнера для идеала <f_list>.
    Использует критерий взаимной простоты для пропуска ненужных S-полиномов.
    """
    # Убираем нулевые полиномы
    G = [f.copy() for f in f_list if not f.is_zero()]
    if not G:
        return [RingElement()]

    # Инициализация списка пар
    pairs = [(i, j) for i, j in combinations(range(len(G)), 2)]

    while pairs:
        i, j = pairs.pop(0)
        if i >= len(G) or j >= len(G):
            continue
        f, g = G[i], G[j]

        # Оптимизация: критерий взаимной простоты
        if coprime_criterion_applies(f, g, order_key):
            continue

        # Вычисление и редукция S-полинома
        S = S_polynomial(f, g, order_key)
        r = normal_form(S, G, order_key)

        if not r.is_zero():
            # Нормализация: делаем старший коэффициент равным 1
            lt = leading_term(r, order_key)
            if lt is not None:
                _, lc = lt
                if abs(lc) > 1e-12:
                    r = RingElement({m: c / lc for m, c in r.terms.items()}, r.n_vars)
            G.append(r)
            # Добавляем новые пары с новым элементом
            k = len(G) - 1
            for idx in range(k):
                pairs.append((idx, k))

    # Приведение к редуцированному базису
    G = [g for g in G if not g.is_zero()]
    # Сортировка по убыванию старших мономов
    G.sort(key=lambda g: leading_monomial(g, order_key), reverse=True)
    reduced_basis = []
    for g in G:
        r = normal_form(g, reduced_basis, order_key)
        lt = leading_term(r, order_key)
        if lt is not None:
            _, lc = lt
            if abs(lc) > 1e-12:
                r = RingElement({m: c / lc for m, c in r.terms.items()}, r.n_vars)
        reduced_basis.append(r)
    return reduced_basis


# Мономиальные порядки

def lex_key(m: Monomial) -> tuple:
    return m

def grlex_key(m: Monomial) -> tuple:
    return (-sum(m), m)


# Тестирование

if True:
    print("Проверка оптимизации: критерий взаимной простоты")

    # Пример 1: взаимно простые старшие мономы → базис уже Грёбнер
    print("\nПример 1: ⟨x², y³, z⟩ в K[x, y, z]")
    f1 = RingElement({(2, 0, 0): 1.0})   # x^2
    f2 = RingElement({(0, 3, 0): 1.0})   # y^3
    f3 = RingElement({(0, 0, 1): 1.0})   # z
    basis = [f1, f2, f3]
    
    # Проверим критерий вручную
    print("Пары:")
    for (i, j) in combinations(range(3), 2):
        applies = coprime_criterion_applies(basis[i], basis[j], lex_key)
        lm_i = leading_monomial(basis[i], lex_key)
        lm_j = leading_monomial(basis[j], lex_key)
        print(f"  ({i+1}, {j+1}): LM = {lm_i}, {lm_j} → критерий применим: {applies}")
    
    G = buchberger(basis, lex_key)
    print("Полученный базис Грёбнера:")
    for i, g in enumerate(G, 1):
        print(f"  g{i} = {g}")
    print("Вывод: базис не изменился — критерий корректно сработал.")

    # Пример 2: не взаимно простые → требуется расширение
    print("\nПример 2: ⟨x^2 - y, xy - 1⟩")
    g1 = RingElement({(2, 0): 1.0, (0, 1): -1.0})   # x^2 - y
    g2 = RingElement({(1, 1): 1.0, (0, 0): -1.0})   # xy - 1
    print("LM(g1) =", leading_monomial(g1, lex_key))
    print("LM(g2) =", leading_monomial(g2, lex_key))
    print("Критерий применим?", coprime_criterion_applies(g1, g2, lex_key))

    G2 = buchberger([g1, g2], lex_key)
    print("Редуцированный базис:")
    for i, g in enumerate(G2, 1):
        print(f"  g{i} = {g}")

    print("\nEnd.")

Проверка оптимизации: критерий взаимной простоты

Пример 1: ⟨x², y³, z⟩ в K[x, y, z]
Пары:
  (1, 2): LM = (2, 0, 0), (0, 3, 0) → критерий применим: True
  (1, 3): LM = (2, 0, 0), (0, 0, 1) → критерий применим: True
  (2, 3): LM = (0, 3, 0), (0, 0, 1) → критерий применим: True
Полученный базис Грёбнера:
  g1 = x1^2
  g2 = x2^3
  g3 = x3
Вывод: базис не изменился — критерий корректно сработал.

Пример 2: ⟨x^2 - y, xy - 1⟩
LM(g1) = (2, 0)
LM(g2) = (1, 1)
Критерий применим? False
Редуцированный базис:
  g1 = x1^2 - x2
  g2 = x1x2 - 1
  g3 = -x2^2 + x1
  g4 = x2^3 - 1

End.


---

**Задача (Expert): Напишите функцию проверки полинома на верность королю**

In [3]:
def is_in_ideal(f: RingElement, G: List[RingElement], order_key) -> bool:
    """
    Проверяет, принадлежит ли f идеалу ⟨G⟩.
    G должен быть редуцированным базисом Грёбнера (нормализованным: LT = LM).
    """
    nf = normal_form(f, G, order_key)
    return nf.is_zero()

---


**Задача (Expert): Решите несколько систем нелинейных алгебраических уравнений от двух и более переменных с помощью построения базиса Грёбнера. Выполните для каждой
системы:**
1. Выберите удобный порядок мономов (обоснуйте выбор).
2. Постройте (вручную для простых примеров) или с помощью программы базис Грёбнера.
3. Получите последовательность уравнений для отдельных
переменных (исключение переменных).
4. Сравните результаты с выводом CAS (Wolfram|Alpha,
Sage, Singular, SymPy): проверяйте эквивалентность идеалов и наборы корней.
5. Оцените вычислительную сложность и опишите, какие
оптимизации алгоритма Бухбергера помогли бы ускорить
расчёт для данной системы.


## 🔹 Пример 1: $ \begin{cases} x^2 + y^2 = 1 \\ x - y = 0 \end{cases} $

### 1. Обоснование выбора порядка  
Выбираем **лексикографический порядок** с $x > y$ (обозначим $\succ_{\text{lex}}$), поскольку:
- $\succ_{\text{lex}}$ обладает *свойством элиминации*: если $G$ — базис Грёбнера относительно $\succ_{\text{lex}}$ с $x > y$, то $G \cap \mathbb{Q}[y]$ порождает элиминационный идеал $I \cap \mathbb{Q}[y]$.
- Это позволяет получить **треугольную систему**: сначала уравнение от $y$, затем — выражение $x$ через $y$.

### 2. Построение базиса (вручную)

Идеал:  
$$
I = \langle f_1, f_2 \rangle = \langle x^2 + y^2 - 1,\; x - y \rangle.
$$

Вычислим $S$-полином:
$$
\operatorname{LM}(f_1) = x^2,\quad \operatorname{LM}(f_2) = x,\quad \operatorname{LCM}(x^2, x) = x^2,
$$
$$
S(f_1,f_2) = \frac{x^2}{x^2}f_1 - \frac{x^2}{x}f_2 = (x^2 + y^2 - 1) - x(x - y) = xy + y^2 - 1.
$$
Редукция по $f_2$ ($x \to y$):
$$
xy + y^2 - 1 \xrightarrow{x \mapsto y} y^2 + y^2 - 1 = 2y^2 - 1.
$$
Нормализуем (ведущий коэффициент = 1):
$$
g_2 = y^2 - \tfrac12.
$$

Тогда редуцированный базис:
$$
G = \bigl\{\, x - y,\;\; y^2 - \tfrac12 \,\bigr\}.
$$

### 3. Исключение переменных  
Из $G$:
$$
y^2 = \frac12 \;\Rightarrow\; y = \pm \frac{1}{\sqrt{2}}, \qquad x = y.
$$
Решения:
$$
\left( \frac{\sqrt{2}}{2},\; \frac{\sqrt{2}}{2} \right),\quad
\left( -\frac{\sqrt{2}}{2},\; -\frac{\sqrt{2}}{2} \right).
$$

### 4. Проверка  
- В **SymPy**/ **Wolfram|Alpha** — те же корни.
- Проверка принадлежности идеалу: $\;f_1,f_2 \in \langle G \rangle$ и $g_1,g_2 \in \langle f_1,f_2 \rangle$ — верно.

### 5. Оптимизации  
- Критерий взаимной простоты **не применим** ($\operatorname{LM}(f_1) = x^2$, $\operatorname{LM}(f_2) = x$ — не взаимно просты).
- Однако нормализация и раннее обнаружение нулевой нормальной формы ускоряют редукцию.

In [4]:
# Мономиальный порядок: lex (x > y)
order = lex_key

# Определяем систему
f1 = RingElement({(2, 0): 1.0, (0, 2): 1.0, (0, 0): -1.0})  # x^2 + y^2 - 1
f2 = RingElement({(1, 0): 1.0, (0, 1): -1.0})               # x - y

print("Исходные полиномы:")
print("f1 =", f1)
print("f2 =", f2)

# Строим базис Грёбнера
G = buchberger([f1, f2], order)
print("\nРедуцированный базис Грёбнера G:")
for i, g in enumerate(G, 1):
    print(f"g{i} =", g)

# Проверяем принадлежность
print("\nПроверка принадлежности идеалу ⟨G⟩:")
print("f1 ∈ ⟨G⟩?", is_in_ideal(f1, G, order))
print("f2 ∈ ⟨G⟩?", is_in_ideal(f2, G, order))

# Нормальные формы — должны быть 0
nf1 = normal_form(f1, G, order)
nf2 = normal_form(f2, G, order)
print("NF(f1) =", nf1)
print("NF(f2) =", nf2)

# Решение (вручную по базису)
import math
solutions = [
    ( math.sqrt(2)/2,  math.sqrt(2)/2),
    (-math.sqrt(2)/2, -math.sqrt(2)/2)
]
print("\nРешения (аналитически):")
for sol in solutions:
    print(f"x = {sol[0]: .4f}, y = {sol[1]: .4f}")

# Сверяем с SymPy (если установлен)
try:
    from sympy import symbols, solve
    x, y = symbols('x y')
    sym_solutions = solve([x**2 + y**2 - 1, x - y], (x, y))
    print("\nРешения (SymPy):")
    for sol in sym_solutions:
        print(sol)
except Exception as e:
    print("\nSymPy не доступен:", e)

Исходные полиномы:
f1 = x2^2 + x1^2 - 1
f2 = -x2 + x1

Редуцированный базис Грёбнера G:
g1 = x2^2 + x1^2 - 1
g2 = -x2 + x1
g3 = x2^2 - 0.5

Проверка принадлежности идеалу ⟨G⟩:
f1 ∈ ⟨G⟩? True
f2 ∈ ⟨G⟩? True
NF(f1) = 0
NF(f2) = 0

Решения (аналитически):
x =  0.7071, y =  0.7071
x = -0.7071, y = -0.7071

Решения (SymPy):
(-sqrt(2)/2, -sqrt(2)/2)
(sqrt(2)/2, sqrt(2)/2)
